In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
import random
import string
from spotipy import SpotifyClientCredentials, util
import os

# Access Spotify API

In [7]:
manager = SpotifyClientCredentials(os.environ.get("SPOTIPY_CLIENT_ID"),os.environ.get("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(client_credentials_manager=manager)

In [3]:
sp.audio_features('0VjIjW4GlUZAMYd2vXMi3b')

[{'danceability': 0.514,
  'energy': 0.73,
  'key': 1,
  'loudness': -5.934,
  'mode': 1,
  'speechiness': 0.0598,
  'acousticness': 0.00146,
  'instrumentalness': 9.54e-05,
  'liveness': 0.0897,
  'valence': 0.334,
  'tempo': 171.005,
  'type': 'audio_features',
  'id': '0VjIjW4GlUZAMYd2vXMi3b',
  'uri': 'spotify:track:0VjIjW4GlUZAMYd2vXMi3b',
  'track_href': 'https://api.spotify.com/v1/tracks/0VjIjW4GlUZAMYd2vXMi3b',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0VjIjW4GlUZAMYd2vXMi3b',
  'duration_ms': 200040,
  'time_signature': 4}]

# Generate Random Songs
Here, we will need to develop some sort of method to query random songs from the Spotify API for classification

In [8]:
choices = string.ascii_letters + string.digits
choices

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

In [14]:
random.choice(choices)

'I'

In [47]:
q = random.choice(choices) + '%'
item = sp.search(q, limit = 3, type= "track")
for song in item['tracks']['items']:
    print(song['id'])

q%
0iFROFTyditGLmb4CtqMBb
7a53HqqArd4b9NF4XAmlbI
5wOQaoVaJGcWlcowehDjhX


In [10]:
def getRandomSongIDS(n):
    """
    params:
    n = number of random songs to generate
    returns:
    a list of random song IDs
    """
    choices = string.ascii_letters + string.digits
    returnList = []
    for i in range(n // 3):
        q = random.choice(choices) + '%'
        item = sp.search(q, limit = 3, type= "track", offset = random.randint(1,1000))
        for song in item['tracks']['items']:
            returnList.append({"trackName": song['name'], "artist": song['artists'][0]['name'], "id": song['id']})
    return returnList
songs = getRandomSongIDS(1000)

# Get Lyrics for each song

In [4]:
import lyricsgenius
genius = lyricsgenius.Genius()
genius.search_song("Waiting On ther World to Change", "John Mayer").lyrics

Searching for "Waiting On ther World to Change" by John Mayer...
Done.


"[Intro]\nOne, two\nOne, two, three\n\n[Verse 1]\nMe and all my friends\nWe're all misunderstood\nThey say we stand for nothing and\nThere's no way we ever could\nNow we see everything that's going wrong\nWith the world and those who lead it\nWe just feel like we don't have the means\nTo rise above and beat it\n\n[Chorus]\nSo we keep waiting (Waiting)\nWaiting on the world to change\nWe keep on waiting (Waiting)\nWaiting on the world to change\nIt's hard to beat the system\nWhen we're standing at a distance\nSo we keep waiting (Waiting)\nWaiting on the world to change\n\n[Verse 2]\nNow if we had the power\nTo bring our neighbors home from war\nThey would have never missed a Christmas\nNo more ribbons on their door\nBut when you trust your television\nWhat you get is what you got\nCause when they own the information, oh\nThey can bend it all they want\n[Chorus]\nThat's why we're waiting (Waiting)\nWaiting on the world to change\nWe keep on waiting (Waiting)\nWaiting on the world to chan

In [18]:
def genDF(songDict):
    df = pd.DataFrame(songDict)
    df = df.apply(lambda X: getSongStats(X), axis = 1)
    def get_lyrics(X):
        try:
            r = genius.search_song(X['trackName'], X['artist']).lyrics
        except:
            return None
        return r
    df['lyrics'] = df.apply(get_lyrics, axis = 1)
    return df
def getSongStats(df):
    try:
        audioFeats = sp.audio_features(df['id'])
        featNames = list(audioFeats[0].keys())[:12]
        returnList = []
        for feat in featNames:
            # returnList.append(audioFeats[0][feat])
            df[feat] = audioFeats[0][feat]
    except AttributeError:
        return None
    return df

In [13]:
h = sp.audio_features('0VjIjW4GlUZAMYd2vXMi3b')
list(h[0].keys())[:12]

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type']

In [19]:
df = genDF(songs)
df

eber...
Done.
Searching for "Peaches" by Justin Bieber...
Done.
Searching for "t h e . c l i m b . b a c k" by J. Cole...
Done.
Searching for "Wish I Knew You" by The Revivalists...
Done.
Searching for "Man! I Feel Like A Woman!" by Shania Twain...
Done.
Searching for "Run It! (feat. Juelz Santana)" by Chris Brown...
Done.
Searching for "Since U Been Gone" by Kelly Clarkson...
Done.
Searching for "Time Today" by Moneybagg Yo...
Done.
Searching for "Hotline Bling" by Drake...
Done.
Searching for "bad idea!" by girl in red...
Done.
Searching for "What Hurts The Most" by Rascal Flatts...
Done.
Searching for "Rain Is A Good Thing" by Luke Bryan...
Done.
Searching for "Down Bad (feat. Millie Go Lightly)" by Real Recognize Rio...
Done.
Searching for "Formation" by Beyoncé...
Done.
Searching for "Peligrosa" by J Balvin...
Done.
Searching for "I Won't Give Up" by Jason Mraz...
Done.
Searching for "After The Storm (feat. Tyler, The Creator & Bootsy Collins)" by Kali Uchis...
Done.
Searching for

,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics
0,Tu Principe - Live,Daddy Yankee,2cLimke50rVKfaNi67Pgdw,0.750,0.958,1.0,-2.802,0.0,0.1230,0.11600,0.000000,0.7230,0.491,94.002,audio_features,Source/Fuente1. GRABACIÓN DEL AÑO\nRubén Blade...
1,Vuelve,Daddy Yankee,4xoVC7kbmCj5R8v0G1yd31,0.734,0.531,1.0,-5.662,1.0,0.0321,0.19600,0.000000,0.1190,0.199,130.025,audio_features,"[Letra de ""Vuelve"" ft. Bad Bunny]\n\n[Intro: B..."
2,Pasarela,Daddy Yankee,7CFagBkEiUN779A3IoZHqI,0.839,0.907,9.0,-4.259,1.0,0.0453,0.10100,0.000000,0.3320,0.737,121.048,audio_features,"[Letra de ""Pasarela""]\n\n[Intro]\nBoom bap\nMo..."
3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,[Spoken Word: Alan Watts]\nA person who thinks...
4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,[Instrumental Intro]\n\n[Verse 1]\nAlways see ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,90mh,Trefuego,1VxvGm1moDJ3svQlwjdBwA,0.716,0.427,1.0,-8.993,1.0,0.0529,0.18700,0.000000,0.3540,0.223,108.993,audio_features,"[Intro]\n(Hi Kevin)\nAyy, yeah, ayy\nAyy\n\n[C..."
995,9 Bridge,Rowdy Rebel,2sHekv6OdEiO4htSjdB9j4,0.642,0.589,5.0,-7.392,0.0,0.3020,0.12400,0.000000,0.0868,0.720,95.543,audio_features,[Intro: A Boogie wit da Hoodie]\nI know I don'...
996,Lotus Flower Bomb (feat. Miguel),Wale,3MAgQuClHcAV8E9CbeBS6f,0.512,0.598,9.0,-4.959,0.0,0.1150,0.61100,0.000000,0.0881,0.345,70.189,audio_features,[Spoken: Wale]\nI'ma rap to you real quick\nI ...
997,Envy Me,Calboy,7rvyVWja33WG9R97oeJAjx,0.740,0.488,1.0,-7.664,0.0,0.2700,0.23400,0.000000,0.2410,0.584,149.042,audio_features,[Chorus]\nNow I lay me down to sleep (Now I la...


In [24]:
df2 = df[~df['lyrics'].isna()]

In [21]:
df.to_csv("trainingSongs.csv")

In [15]:
df = df[~df['lyrics'].isna()]
df

,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics
1,NEED YOU MOST (So Sick),The Kid LAROI,2DsdZx34mT4jVtnPk1eoFM,0.630,0.707,5,-3.632,1,0.0508,0.06810,0.000000,0.2840,0.2850,86.849,audio_features,\n\n[Verse 1]\nWe fight every day and night\nY...
2,Hard For,Kevin Gates,0YSTRuZ7elW9CEBfZJW5Js,0.716,0.485,8,-8.241,1,0.2370,0.04010,0.000000,0.2770,0.5560,148.141,audio_features,\n\n[Intro]\nMan there ain't no sense in playi...
3,JUST LIKE YOU,NF,7tTRFVlFzk6yTnoMcParmi,0.775,0.757,9,-4.607,0,0.2280,0.48900,0.000000,0.2800,0.5410,140.044,audio_features,\n\n[Chorus]\nJust in case my car goes off the...
5,WITHOUT YOU (with Miley Cyrus),The Kid LAROI,3vgnToBTuzVl2Xp53m0QcS,0.442,0.465,0,-6.283,1,0.0427,0.18500,0.000000,0.0949,0.4150,93.109,audio_features,"\n\n[Intro: The Kid LAROI]\nYeah, yeah, yeah\n..."
7,Somebody Like That,Tenille Arts,3hboOw4dS0aXGuP0fyyvbC,0.574,0.798,6,-5.356,1,0.0485,0.01520,0.000000,0.1110,0.5800,105.034,audio_features,\n\n[Verse 1]\nI've seen pick up lines and div...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,Wet Dreamz,J. Cole,6Ius4TC0L3cN74HT7ENE6e,0.517,0.705,6,-8.205,0,0.3640,0.07510,0.000000,0.1280,0.5840,175.906,audio_features,\n\n[Intro]\nCole\nCole world\nYeah\nLet me ta...
35,Wishing Well,Juice WRLD,2U5WueTLIK5WJLD7mvDODv,0.650,0.613,9,-6.130,0,0.1280,0.00336,0.000000,0.2670,0.0804,149.972,audio_features,"\n\n[Intro]\nMm-mm, mm-mm-mm-mm, mm-mm-mm, uh\..."
41,Where The Hood At,DMX,6RUpJeXT6U4SPEafnr9Wr7,0.537,0.948,8,-3.063,1,0.3550,0.15100,0.000000,0.3560,0.7920,186.151,audio_features,\n\n[Intro]\n(Now I gotta deal with this shit)...
42,Boys Will Be Bugs,Cavetown,6suYoN4gFkanqOBn5yRnJC,0.486,0.528,1,-11.722,1,0.0463,0.00387,0.000014,0.1060,0.4520,178.519,audio_features,\n\n[Verse 1]\nI'm a dumb teen boy\nI eat stic...


# Generate Clusters
Though there are many variables for each song that we could develop clusters with, we chose only 4.

In [31]:
from sklearn.cluster import KMeans

kmeans = KMeans(8)
X = df2[["energy", "valence", "tempo", "liveness"]]
kmeans.fit_transform(X)
df2['cluster'] = kmeans.labels_
df2

<ipython-input-31-fb781f81bdf4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cluster'] = kmeans.labels_


,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster
0,Tu Principe - Live,Daddy Yankee,2cLimke50rVKfaNi67Pgdw,0.750,0.958,1.0,-2.802,0.0,0.1230,0.11600,0.000000,0.7230,0.491,94.002,audio_features,Source/Fuente1. GRABACIÓN DEL AÑO\nRubén Blade...,1
1,Vuelve,Daddy Yankee,4xoVC7kbmCj5R8v0G1yd31,0.734,0.531,1.0,-5.662,1.0,0.0321,0.19600,0.000000,0.1190,0.199,130.025,audio_features,"[Letra de ""Vuelve"" ft. Bad Bunny]\n\n[Intro: B...",0
2,Pasarela,Daddy Yankee,7CFagBkEiUN779A3IoZHqI,0.839,0.907,9.0,-4.259,1.0,0.0453,0.10100,0.000000,0.3320,0.737,121.048,audio_features,"[Letra de ""Pasarela""]\n\n[Intro]\nBoom bap\nMo...",0
3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,[Spoken Word: Alan Watts]\nA person who thinks...,0
4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,[Instrumental Intro]\n\n[Verse 1]\nAlways see ...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,90mh,Trefuego,1VxvGm1moDJ3svQlwjdBwA,0.716,0.427,1.0,-8.993,1.0,0.0529,0.18700,0.000000,0.3540,0.223,108.993,audio_features,"[Intro]\n(Hi Kevin)\nAyy, yeah, ayy\nAyy\n\n[C...",3
995,9 Bridge,Rowdy Rebel,2sHekv6OdEiO4htSjdB9j4,0.642,0.589,5.0,-7.392,0.0,0.3020,0.12400,0.000000,0.0868,0.720,95.543,audio_features,[Intro: A Boogie wit da Hoodie]\nI know I don'...,1
996,Lotus Flower Bomb (feat. Miguel),Wale,3MAgQuClHcAV8E9CbeBS6f,0.512,0.598,9.0,-4.959,0.0,0.1150,0.61100,0.000000,0.0881,0.345,70.189,audio_features,[Spoken: Wale]\nI'ma rap to you real quick\nI ...,4
997,Envy Me,Calboy,7rvyVWja33WG9R97oeJAjx,0.740,0.488,1.0,-7.664,0.0,0.2700,0.23400,0.000000,0.2410,0.584,149.042,audio_features,[Chorus]\nNow I lay me down to sleep (Now I la...,5


In [36]:
n = 7
df2[df2['cluster'] == n]

,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster
283,Soft Brown Noise,High Altitude Samples,129ldjjvnFfA7Ygc8ktYnh,0.0,0.00148,5.0,-31.839,1.0,0.0,0.394,1.0,0.111,0.0,0.0,audio_features,FINAL\nLONE SURVIVOR\nWritten by Peter Berg\nB...,7


In [29]:
df2.groupby("cluster")[["energy", "valence", "tempo", "liveness"]].mean()

,energy,valence,tempo,liveness
cluster,,,,
0,0.702170,0.570808,95.914519,0.187937
1,0.610500,0.459057,140.092103,0.163713
2,0.718260,0.555446,179.146042,0.160161
3,0.673601,0.542928,110.661055,0.187163
4,0.514549,0.446085,78.362480,0.174883
5,0.703432,0.502443,159.374682,0.174491
6,0.669611,0.538556,124.810765,0.183728
7,0.001480,0.000000,0.000000,0.111000


# Clean Lyrics

In [39]:
import re

## Eliminate escape characters and trim down white space

In [127]:
def stringProcessing(s):
    s = re.sub(r"\'", "", s)
    s = re.sub(r'\n', ' ', s)
    s = re.sub(r"\[[^[]*\]", '', s)
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = s.strip()
    s = s.lower()
    return s

In [131]:
df2['lyrics'] = df2['lyrics'].apply(stringProcessing)

<ipython-input-131-31c2179d9317>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lyrics'] = df2['lyrics'].apply(stringProcessing)


In [132]:
df2

,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster
0,Tu Principe - Live,Daddy Yankee,2cLimke50rVKfaNi67Pgdw,0.750,0.958,1.0,-2.802,0.0,0.1230,0.11600,0.000000,0.7230,0.491,94.002,audio_features,source fuente1 grabación del año rubén blades ...,1
1,Vuelve,Daddy Yankee,4xoVC7kbmCj5R8v0G1yd31,0.734,0.531,1.0,-5.662,1.0,0.0321,0.19600,0.000000,0.1190,0.199,130.025,audio_features,tengo miedo de morirme y volver a nacer y que ...,0
2,Pasarela,Daddy Yankee,7CFagBkEiUN779A3IoZHqI,0.839,0.907,9.0,-4.259,1.0,0.0453,0.10100,0.000000,0.3320,0.737,121.048,audio_features,boom bap modelo uno bótate modelo dos bótate m...,0
3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,a person who thinks all the time has nothing t...,0
4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,always see it on t v or read in the magazines ...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,90mh,Trefuego,1VxvGm1moDJ3svQlwjdBwA,0.716,0.427,1.0,-8.993,1.0,0.0529,0.18700,0.000000,0.3540,0.223,108.993,audio_features,hi kevin ayy yeah ayy ayy you dont really want...,3
995,9 Bridge,Rowdy Rebel,2sHekv6OdEiO4htSjdB9j4,0.642,0.589,5.0,-7.392,0.0,0.3020,0.12400,0.000000,0.0868,0.720,95.543,audio_features,i know i dont never make promises this time i ...,1
996,Lotus Flower Bomb (feat. Miguel),Wale,3MAgQuClHcAV8E9CbeBS6f,0.512,0.598,9.0,-4.959,0.0,0.1150,0.61100,0.000000,0.0881,0.345,70.189,audio_features,ima rap to you real quick i wanna enjoy the lu...,4
997,Envy Me,Calboy,7rvyVWja33WG9R97oeJAjx,0.740,0.488,1.0,-7.664,0.0,0.2700,0.23400,0.000000,0.2410,0.584,149.042,audio_features,now i lay me down to sleep now i lay me down t...,5


## Detect Language

In [133]:
from langdetect import detect

In [135]:
df2['language'] = df2['lyrics'].apply(detect)
df2

<ipython-input-135-406ef6c809e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['language'] = df2['lyrics'].apply(detect)


,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster,language
0,Tu Principe - Live,Daddy Yankee,2cLimke50rVKfaNi67Pgdw,0.750,0.958,1.0,-2.802,0.0,0.1230,0.11600,0.000000,0.7230,0.491,94.002,audio_features,source fuente1 grabación del año rubén blades ...,1,es
1,Vuelve,Daddy Yankee,4xoVC7kbmCj5R8v0G1yd31,0.734,0.531,1.0,-5.662,1.0,0.0321,0.19600,0.000000,0.1190,0.199,130.025,audio_features,tengo miedo de morirme y volver a nacer y que ...,0,es
2,Pasarela,Daddy Yankee,7CFagBkEiUN779A3IoZHqI,0.839,0.907,9.0,-4.259,1.0,0.0453,0.10100,0.000000,0.3320,0.737,121.048,audio_features,boom bap modelo uno bótate modelo dos bótate m...,0,es
3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,a person who thinks all the time has nothing t...,0,en
4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,always see it on t v or read in the magazines ...,3,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,90mh,Trefuego,1VxvGm1moDJ3svQlwjdBwA,0.716,0.427,1.0,-8.993,1.0,0.0529,0.18700,0.000000,0.3540,0.223,108.993,audio_features,hi kevin ayy yeah ayy ayy you dont really want...,3,en
995,9 Bridge,Rowdy Rebel,2sHekv6OdEiO4htSjdB9j4,0.642,0.589,5.0,-7.392,0.0,0.3020,0.12400,0.000000,0.0868,0.720,95.543,audio_features,i know i dont never make promises this time i ...,1,en
996,Lotus Flower Bomb (feat. Miguel),Wale,3MAgQuClHcAV8E9CbeBS6f,0.512,0.598,9.0,-4.959,0.0,0.1150,0.61100,0.000000,0.0881,0.345,70.189,audio_features,ima rap to you real quick i wanna enjoy the lu...,4,en
997,Envy Me,Calboy,7rvyVWja33WG9R97oeJAjx,0.740,0.488,1.0,-7.664,0.0,0.2700,0.23400,0.000000,0.2410,0.584,149.042,audio_features,now i lay me down to sleep now i lay me down t...,5,en


In [138]:
ready = df2[df2['language'] == 'en']

In [140]:
ready.to_csv("trainingSongs_clean.csv")

In [141]:
ready.iloc[2]["lyrics"]

'baby all i got is this beat up leather bag and everything i own dont fill up half but dont you worry about the way i pack all i care about is getting back real soon a goodbye kiss is all i need from you cause im carrying your love with me west virginia down to tennessee ill be moving with the good lords speed carrying your love with me its my strength for holding on every minute that i have to be gone ill have everything i ever need carrying your love with me on a lonely highway stuck out in the rain darling all i have to do is speak your name the clouds roll back and the waters part the sun starts shining in my heart for you youre right there in everything i do cause im carrying your love with me west virginia down to tennessee ill be moving with the good lords speed carrying your love with me its my strength for holding on every minute that i have to be gone ill have everything ill ever need im carrying your love with me im carrying your love with me west virginia down to tennessee 